In [15]:
# Our image frag and shuffle worked out. Time to import that work here and build a starter
# reconstruction algo (we'll refine the strategy after a simple, naive-ish approach)

# Importing tools
from PIL import Image, ImageFilter
import numpy as np
from numpy import random
import math

# Reading image from .jpg data
im = Image.open( 'TestImages/flowers.jpg' ).rotate(-90)

# Optional original image visualization in-notebook
#im

In [16]:
# converting image data to numpy array
pix = np.array(im);

# We've noticed that the rotated (kitty trial) image has a padded black border,
# which we want to remove (hard-coded here) for reconstruction (artificial original
# border = ability to 'cheat' the reconstruction algo!)
RL = 0
RU = 3024
CL = 550
CU = 3500
pixCropped = pix[RL:RU,CL:CU,0:3]

# optional image conversion and printing of cropped image data
imCropped = Image.fromarray(pixCropped,'RGB')
#imCropped

In [17]:
# Now we want to let the user/test administrator pick a number of
# row partitions and column partitions for fragging the image.

# test administrator-defined partitions
rowPartitions = 5
colPartitions = 3

# storing values for # total rows, cols (pixels)
pixelRows = pixCropped.shape[0];
pixelCols = pixCropped.shape[1];

# row, column length of partitions
rowInterval = math.ceil(pixelRows/rowPartitions);
colInterval = math.ceil(pixelCols/colPartitions);

#(optional) preparing and printing a preliminary impression of partitioning
pixPartitioned1 = np.copy(pixCropped)

# row partitions
for i in range(1,rowPartitions):
    rowBegin = i*rowInterval-4
    rowEnd   = i*rowInterval+5
    pixPartitioned1[rowBegin:rowEnd,:,:] = 255
        
# column partitions
for j in range(1,colPartitions):
    colBegin = j*colInterval-4
    colEnd   = j*colInterval+5
    pixPartitioned1[:,colBegin:colEnd,:] = 255

# printing out a concept illustration of the fragmentation
imPartitioned1 = Image.fromarray(pixPartitioned1)
#imPartitioned1

In [18]:
# determining number of total image fragments based on user input
numFragments = rowPartitions*colPartitions

# we can shuffle these flattened indices with a random permutation:
shufFragIndices = np.random.permutation(numFragments)

# now we'll define our overlap and offsets.
offsetMax = 2

overlapPix = 10

# ensuring overlap algo will work (sufficient overlap for offsets)
overlapPix = max(overlapPix,3*offsetMax)

# using these parameters, we'll define an array of x,y pixel
# offsets for each image fragment
xyOffsets = np.random.randint(2*offsetMax+1,size=(numFragments,2))-offsetMax
#xyOffsets

In [19]:
# We'll initialize and fill a 4-D (!) array to handle all the frags in the shuffled
# order given by shufFragIndices.
pixFrags = np.zeros((numFragments,rowInterval + 2*overlapPix,colInterval + 2*overlapPix,3),dtype='uint8')

for i in range(0,numFragments):
    # block indices for starting fragmentation
    blockRowInd = shufFragIndices[i] // colPartitions
    blockColInd = shufFragIndices[i] % colPartitions
    
    # defining part of original image given over to fragment 'i'
    # (including overlap and x/y offset)
    startRow = blockRowInd*rowInterval - overlapPix + xyOffsets[i,0]
    endRow = startRow + rowInterval + 2*overlapPix
    startCol = blockColInd*colInterval - overlapPix + xyOffsets[i,1]
    endCol = startCol + colInterval + 2*overlapPix
    
    # culling interval back in the case of original image overflow/underflow
    if startRow < 0:
        startRow = abs(xyOffsets[i,0])
        endRow = rowInterval + 2*overlapPix + abs(xyOffsets[i,0])
    elif endRow > pixCropped.shape[0]:
        startRow = pixCropped.shape[0] - rowInterval - 2*overlapPix - abs(xyOffsets[i,0])
        endRow = pixCropped.shape[0] - abs(xyOffsets[i,0])
        
    if startCol < 0:
        startCol = abs(xyOffsets[i,1])
        endCol = colInterval + 2*overlapPix + abs(xyOffsets[i,1])
    elif endCol > pixCropped.shape[1]:
        startCol = pixCropped.shape[1] - colInterval - 2*overlapPix - abs(xyOffsets[i,1])
        endCol = pixCropped.shape[1] - abs(xyOffsets[i,1])
    
    # now we fill in pixFrags (preallocated shape) with shuffled and offset data from
    # pixCropped.
    pixFrags[i,:,:,:] = pixCropped[startRow:endRow,startCol:endCol,:]

In [20]:
# (Optional) We can now assemble a shuffled view of the original photo!
pixShuffled = np.zeros((rowInterval*rowPartitions,colInterval*colPartitions,3),dtype = 'uint8')

for i in range(0,numFragments):
    blockRowInd = i // colPartitions
    blockColInd = i % colPartitions
    
    startRow = rowInterval*blockRowInd
    endRow = startRow + rowInterval
    
    startCol = colInterval*blockColInd
    endCol = startCol + colInterval
     
    fragRS = overlapPix
    fragRE = rowInterval + overlapPix
    
    fragCS = overlapPix
    fragCE = colInterval + overlapPix
    
    pixShuffled[startRow:endRow,startCol:endCol,:] = pixFrags[i,fragRS:fragRE,fragCS:fragCE,:]
    
imShuffled1 = Image.fromarray(pixShuffled)
#imShuffled1

In [21]:
# Here begins the first reconstruction algorithm!

# Since we start with the shuffled collection of subimage data
# in the 4-D array of pixFrags, and since we start with no idea
# of how they relate, we'll need to initialize and then work toward
# filling out a graph of relationships between the different images.

# The assumption we'll operate under during this project is that, as
# we've set the images up, the graph we'll need is very simple and rigidly-
# structured...each image points to LEFT, RIGHT, ABOVE, and BELOW neighbors
# (with the exception of edge and corner cases).  We'll also have to account
# for a random offset relative to each neighbor.

In [22]:
# Key to the graph: for each subimage 'i', we'll store a 4 x 3 x 3 array of
# neighbor data. Out of the four 3-tuples, the SECOND index will point to:

# [i,0,0] contains the ith image's LEFT
#         neighbor's FIRST index in pixFrags.
# [i,1,0] contains the ith image's RIGHT
#         neighbor's FIRST index in pixFrags.
# [i,2,0] contains the ith image's ABOVE
#         neighbor's FIRST index in pixFrags.
# [i,3,0] contains the ith image's BELOW
#         neighbor's FIRST index in pixFrags.

# The second set of tuples ([i,0,1],[i,1,1],[i,2,1],[i,3,1])
# will contain X-OFFSET data (relative to the respective neighbor).

# The third set of tuples ([i,0,2],[i,1,2],[i,2,2],[i,3,2])
# will contain Y-OFFSET data (relative to the respective neighbor).

# We'll initialize everything to -1 (a negative flag indicates no
# matches have yet been made on the corresponding side).

# Below, we'll compute a true graph of where all relationships SHOULD
# end up in the reconstruction graph (DEBUGGING/ANALYTICS PURPOSES ONLY)
trueGraph = -1*np.ones((numFragments,4))
for i in range(0,numFragments):
    tempRowIdx = shufFragIndices[i] // colPartitions
    tempColIdx = shufFragIndices[i] % colPartitions
    
    # determining left neighbor in original (and then shuffled) graph
    leftColIdx = tempColIdx-1
    if leftColIdx >= 0:
        leftNeighborIdx = tempRowIdx*colPartitions + leftColIdx
        for j in range(0,numFragments):
            if shufFragIndices[j] == leftNeighborIdx: 
                trueGraph[i,0] = j

    # determining right neighbor in original (and then shuffled) graph
    rightColIdx = tempColIdx+1
    if rightColIdx < colPartitions:
        rightNeighborIdx = tempRowIdx*colPartitions + rightColIdx
        for j in range(0,numFragments):
            if shufFragIndices[j] == rightNeighborIdx:
                trueGraph[i,1] = j

    # determining upper neighbor in original (and then shuffled) graph
    upperRowIdx = tempRowIdx-1
    if upperRowIdx >= 0:
        upperNeighborIdx = upperRowIdx*colPartitions + tempColIdx
        for j in range(0,numFragments):
            if shufFragIndices[j] == upperNeighborIdx:
                trueGraph[i,2] = j

    # determining lower neighbor in original (and then shuffled) graph
    lowerRowIdx = tempRowIdx+1
    if lowerRowIdx < rowPartitions:
        lowerNeighborIdx = lowerRowIdx*colPartitions + tempColIdx
        for j in range(0,numFragments):       
            if shufFragIndices[j] == lowerNeighborIdx:
                trueGraph[i,3] = j
        
#print(trueGraph)

In [23]:
# Okay! How do we start filling out this graph? For a naive first approach,
# we can simply "go down the line" in pixFrags, looking for matches in all the
# other subimages.

In [24]:
%%time

fragmentGraph = -1*np.ones((numFragments,4,3))

###############################################
### DETERMINING LEFT -> RIGHT RELATIONSHIPS ###
###############################################

fragsDim = pixFrags.shape
fragsHeight = fragsDim[1]
fragsWidth  = fragsDim[2]

for i in range(0,numFragments):
    # starting by designating a current "pivot" position
    # where we're determining key graph relationships
    pivotFrag = pixFrags[i,:,:,:]
    
    for j in range(0,numFragments):
        # don't want to compare to self!
        
        bigBreak = False
        
        if j != i:
            # let's begin in the middle of the two images and
            # try tentative offsets/matching data subsequences from there.
            queryFrag = pixFrags[j,:,:,:]
            
            # for this preliminary trial, we'll pivot on the outermost
            # rows and columns of pivotFrag's data, probing for matching
            # subsequences in queryFrag (where we'll have to look through
            # a non-trivial range of columns and rows (due to overlap/offset))
            
            # declaring bounds for subsequence search
            maxMatchWidth = colInterval//2
            maxMatchHeight = rowInterval//2
            
            maxWidthPush = maxMatchWidth//2
            maxHeightPush = maxMatchHeight//2
            
            # let's start comparing the right side of pivotFrag
            # with the left side of queryFrag.
            midRowPivot = fragsHeight//2
            midColPivot = fragsWidth//2
            
            # looking for small comparison subsets at first (then building out to confirm)
            midPivotMat = (pivotFrag[midRowPivot-3:midRowPivot+3,fragsWidth-1,:]
                           .astype('int32'))
            bigPivotMat = (pivotFrag[midRowPivot-maxHeightPush:midRowPivot+maxHeightPush,
                                     fragsWidth-1,:]).astype('int32')
            
            tunnelDepth = 3
            tempQueryXIdx = tunnelDepth
            
            # regarding limits of the search: we may have to 'tunnel' as much as
            # 3*overlapPix pixels + 2*offsetMax into the neighbor to find a match due to how
            # these overlaps work near the original image boundary.
            
            # NOTE: the situation is different if we only have 2 column partitions!!!
            #       need to push x all the way to 4*overlapPix!
            xOverlapFactor = 3
            if colPartitions == 2:
                xOverlapFactor = 4
            
            while tempQueryXIdx < xOverlapFactor*overlapPix + 2 + 2*offsetMax:
                
                tempYOffset = -3*offsetMax
                while tempYOffset < 3*offsetMax + 2:
                    
                    qRS = midRowPivot - 3 + tempYOffset
                    qRE = midRowPivot + 3 + tempYOffset

                    # what we're creating here is an initial comparison
                    # sliding window of 7 pixels total (3 up, 3 down).
                    # if these are "close enough" to each other, we'll push
                    # the comparison out to try and verify a match.
                    
                    # let's try and anticipate some possible corruption/bias in our
                    # comparison here.
                    tempQueryMat = queryFrag[qRS:qRE,tempQueryXIdx,:].astype('int32')
                    tempDiffMat = abs(tempQueryMat-midPivotMat)
                    
                    # being in uint8, with 3 channels, let's look for similarity
                    # (7*1*3 = 21 comparison pixel window)
                    if sum(sum(tempDiffMat)) < 10*tempDiffMat.size:
                        # in the case of a close preliminary match, we'll expand the
                        # compare to a much bigger window.
                        qRS = midRowPivot - maxHeightPush + tempYOffset
                        qRE = midRowPivot + maxHeightPush + tempYOffset
                        
                        bigQueryMat = queryFrag[qRS:qRE,tempQueryXIdx,:].astype('int32')    
                        tempDiffMat = abs(bigQueryMat-bigPivotMat)
                        
                        if sum(sum(tempDiffMat)) <= 5*tempDiffMat.size:
                            # in the case of a 'big match' we'll tunnel even further
                            # and see if the match continues.
                            
                            for xTunnel in range(1,tunnelDepth):
                                bigQueryMat = queryFrag[qRS:qRE,tempQueryXIdx-xTunnel,:].astype('int32')
                                bigPivotMatTemp = (pivotFrag[midRowPivot-maxHeightPush:midRowPivot+maxHeightPush,
                                                         fragsWidth-1-xTunnel,:]).astype('int32')
                                tempDiffMat = abs(bigQueryMat-bigPivotMatTemp)
                                
                                if sum(sum(tempDiffMat)) > 2*tempDiffMat.size:
                                    break
                                elif xTunnel == tunnelDepth-1 :
                                    # remember that we're in the left-pivot-to-right-query
                                    # comparison scheme right here.
                                    fragmentGraph[i,1,0] = j
                                    fragmentGraph[i,1,1] = tempQueryXIdx
                                    fragmentGraph[i,1,2] = tempYOffset
                                    bigBreak = True;
                    
                    # if we've found a match, break from the grind in Y and go
                    # to next pivot fragment (this'll take a couple break statements)
                    if(bigBreak):
                        break
                    else:
                        tempYOffset += 1
                
                # if we've found a match, break from the grind in X tunneling
                if(bigBreak):
                    break
                else:
                    tempQueryXIdx += 1
            
            # here's the limit of the i/j frag pair testing (if i != j)
        
        # now, IMPORTANT: if we've found a match and reached here, we
        # DON'T want to break but rather CONTINUE to the next pivot fragment.
        if(bigBreak):
            continue
            
###############################################
### DETERMINING RIGHT -> LEFT RELATIONSHIPS ###
###############################################

for i in range(0,numFragments):
    # starting by designating a current "pivot" position
    # where we're determining key graph relationships
    pivotFrag = pixFrags[i,:,:,:]
    
    for j in range(0,numFragments):
        # don't want to compare to self!
        
        bigBreak = False
        
        if j != i:
            # let's begin in the middle of the two images and
            # try tentative offsets/matching data subsequences from there.
            queryFrag = pixFrags[j,:,:,:]
            
            # for this preliminary trial, we'll pivot on the outermost
            # rows and columns of pivotFrag's data, probing for matching
            # subsequences in queryFrag (where we'll have to look through
            # a non-trivial range of columns and rows (due to overlap/offset))
            
            # declaring bounds for subsequence search
            maxMatchWidth = colInterval//2
            maxMatchHeight = rowInterval//2
            
            maxWidthPush = maxMatchWidth//2
            maxHeightPush = maxMatchHeight//2
            
            # let's start comparing the right side of pivotFrag
            # with the left side of queryFrag.
            midRowPivot = fragsHeight//2
            midColPivot = fragsWidth//2
            
            # looking for small comparison subsets at first (then building out to confirm)
            midPivotMat = (pivotFrag[midRowPivot-3:midRowPivot+3,0,:]
                           .astype('int32'))
            bigPivotMat = (pivotFrag[midRowPivot-maxHeightPush:midRowPivot+maxHeightPush,
                                     0,:]).astype('int32')
            
            tempQueryXIdx = fragsWidth-1-tunnelDepth
            
            # regarding limits of the search: we may have to 'tunnel' as much as
            # 3*overlapPix pixels + 2*offsetMax into the neighbor to find a match due to how
            # these overlaps work near the original image boundary.
            
            # NOTE: the situation is different if we only have 2 column partitions!!!
            #       need to push x all the way to 4*overlapPix!
            xOverlapFactor = 3
            if colPartitions == 2:
                xOverlapFactor = 4
            
            while tempQueryXIdx > fragsWidth-1-(xOverlapFactor*overlapPix + 2 + 2*offsetMax):
                
                tempYOffset = -3*offsetMax
                while tempYOffset < 3*offsetMax + 2:
                    
                    qRS = midRowPivot - 3 + tempYOffset
                    qRE = midRowPivot + 3 + tempYOffset

                    # what we're creating here is an initial comparison
                    # sliding window of 7 pixels total (3 up, 3 down).
                    # if these are "close enough" to each other, we'll push
                    # the comparison out to try and verify a match.
                    
                    # let's try and anticipate some possible corruption/bias in our
                    # comparison here.
                    tempQueryMat = queryFrag[qRS:qRE,tempQueryXIdx,:].astype('int32')
                    tempDiffMat = abs(tempQueryMat-midPivotMat)
                    
                    # being in uint8, with 3 channels, let's look for similarity
                    # (7*1*3 = 21 comparison pixel window)
                    if sum(sum(tempDiffMat)) < 10*tempDiffMat.size:
                        # in the case of a close preliminary match, we'll expand the
                        # compare to a much bigger window.
                        qRS = midRowPivot - maxHeightPush + tempYOffset
                        qRE = midRowPivot + maxHeightPush + tempYOffset
                        
                        bigQueryMat = queryFrag[qRS:qRE,tempQueryXIdx,:].astype('int32')    
                        tempDiffMat = abs(bigQueryMat-bigPivotMat)
                        
                        if sum(sum(tempDiffMat)) <= 5*tempDiffMat.size:
                            # in the case of a 'big match' we'll tunnel even further
                            # and see if the match continues.
                            
                            for xTunnel in range(1,tunnelDepth):
                                bigQueryMat = queryFrag[qRS:qRE,tempQueryXIdx+xTunnel,:].astype('int32')
                                bigPivotMatTemp = (pivotFrag[midRowPivot-maxHeightPush:midRowPivot+maxHeightPush,
                                                         xTunnel,:]).astype('int32')
                                tempDiffMat = abs(bigQueryMat-bigPivotMatTemp)
                                
                                if sum(sum(tempDiffMat)) > 2*tempDiffMat.size:
                                    break
                                elif xTunnel == tunnelDepth-1 :
                                    # remember that we're in the right-pivot-to-left-query
                                    # comparison scheme right here.
                                    fragmentGraph[i,0,0] = j
                                    fragmentGraph[i,0,1] = fragsWidth-tempQueryXIdx
                                    fragmentGraph[i,0,2] = tempYOffset
                                    bigBreak = True;
                    
                    # if we've found a match, break from the grind in Y and go
                    # to next pivot fragment (this'll take a couple break statements)
                    if(bigBreak):
                        break
                    else:
                        tempYOffset += 1
                
                # if we've found a match, break from the grind in X tunneling
                if(bigBreak):
                    break
                else:
                    tempQueryXIdx -= 1
            
            # here's the limit of the i/j frag pair testing (if i != j)
        
        # now, IMPORTANT: if we've found a match and reached here, we
        # DON'T want to break but rather CONTINUE to the next pivot fragment.
        if(bigBreak):
            continue
            
################################################
### DETERMINING ABOVE -> BELOW RELATIONSHIPS ###
################################################

for i in range(0,numFragments):
    # starting by designating a current "pivot" position
    # where we're determining key graph relationships
    pivotFrag = pixFrags[i,:,:,:]
    
    for j in range(0,numFragments):
        # don't want to compare to self!
        
        bigBreak = False
        
        if j != i:
            # let's begin in the middle of the two images and
            # try tentative offsets/matching data subsequences from there.
            queryFrag = pixFrags[j,:,:,:]
            
            # for this preliminary trial, we'll pivot on the outermost
            # rows and columns of pivotFrag's data, probing for matching
            # subsequences in queryFrag (where we'll have to look through
            # a non-trivial range of columns and rows (due to overlap/offset))
            
            # declaring bounds for subsequence search
            maxMatchWidth = colInterval//2
            maxMatchHeight = rowInterval//2
            
            maxWidthPush = maxMatchWidth//2
            maxHeightPush = maxMatchHeight//2
            
            # let's start comparing the right side of pivotFrag
            # with the left side of queryFrag.
            midRowPivot = fragsHeight//2
            midColPivot = fragsWidth//2
            
            # looking for small comparison subsets at first (then building out to confirm)
            midPivotMat = (pivotFrag[fragsHeight-1,midColPivot-3:midColPivot+3,:]
                           .astype('int32'))
            bigPivotMat = (pivotFrag[fragsHeight-1,
                                     midColPivot-maxWidthPush:midColPivot+maxWidthPush,:]).astype('int32')
            
            tempQueryYIdx = tunnelDepth
            
            # regarding limits of the search: we may have to 'tunnel' as much as
            # 3*overlapPix pixels + 2*offsetMax into the neighbor to find a match due to how
            # these overlaps work near the original image boundary.
            
            # NOTE: the situation is different if we only have 2 row partitions!!!
            #       need to push y all the way to 4*overlapPix!
            yOverlapFactor = 3
            if rowPartitions == 2:
                yOverlapFactor = 4
            
            # Note the reversal of roles for x/y offsets here compared to left/right relationships.
            while tempQueryYIdx < yOverlapFactor*overlapPix + 2 + 2*offsetMax:
                
                tempXOffset = -3*offsetMax
                while tempXOffset < 3*offsetMax + 2:
                    
                    qCS = midColPivot - 3 + tempXOffset
                    qCE = midColPivot + 3 + tempXOffset

                    # what we're creating here is an initial comparison
                    # sliding window of 7 pixels total (3 up, 3 down).
                    # if these are "close enough" to each other, we'll push
                    # the comparison out to try and verify a match.
                    
                    # let's try and anticipate some possible corruption/bias in our
                    # comparison here.
                    tempQueryMat = queryFrag[tempQueryYIdx,qCS:qCE,:].astype('int32')
                    tempDiffMat = abs(tempQueryMat-midPivotMat)
                    
                    # being in uint8, with 3 channels, let's look for similarity
                    # (7*1*3 = 21 comparison pixel window)
                    if sum(sum(tempDiffMat)) < 10*tempDiffMat.size:
                        # in the case of a close preliminary match, we'll expand the
                        # compare to a much bigger window.
                        qCS = midColPivot - maxWidthPush + tempXOffset
                        qCE = midColPivot + maxWidthPush + tempXOffset
                        
                        bigQueryMat = queryFrag[tempQueryYIdx,qCS:qCE,:].astype('int32')    
                        tempDiffMat = abs(bigQueryMat-bigPivotMat)
                        
                        if sum(sum(tempDiffMat)) <= 5*tempDiffMat.size:
                            # in the case of a 'big match' we'll tunnel even further
                            # and see if the match continues.
                            
                            for yTunnel in range(1,tunnelDepth):
                                bigQueryMat = queryFrag[tempQueryYIdx-yTunnel,qCS:qCE,:].astype('int32')
                                bigPivotMatTemp = (pivotFrag[fragsHeight-1-yTunnel,
                                                             midColPivot-maxWidthPush:midColPivot+maxWidthPush,
                                                             :]).astype('int32')
                                tempDiffMat = abs(bigQueryMat-bigPivotMatTemp)
                                
                                if sum(sum(tempDiffMat)) > 2*tempDiffMat.size:
                                    break
                                elif yTunnel == tunnelDepth-1 :
                                    # remember that we're in the left-pivot-to-right-query
                                    # comparison scheme right here.
                                    fragmentGraph[i,3,0] = j
                                    fragmentGraph[i,3,1] = tempXOffset
                                    fragmentGraph[i,3,2] = tempQueryYIdx
                                    bigBreak = True;
                    
                    # if we've found a match, break from the grind in Y and go
                    # to next pivot fragment (this'll take a couple break statements)
                    if(bigBreak):
                        break
                    else:
                        tempXOffset += 1
                
                # if we've found a match, break from the grind in X tunneling
                if(bigBreak):
                    break
                else:
                    tempQueryYIdx += 1
            
            # here's the limit of the i/j frag pair testing (if i != j)
        
        # now, IMPORTANT: if we've found a match and reached here, we
        # DON'T want to break but rather CONTINUE to the next pivot fragment.
        if(bigBreak):
            continue
            
################################################
### DETERMINING BELOW -> ABOVE RELATIONSHIPS ###
################################################

for i in range(0,numFragments):
    # starting by designating a current "pivot" position
    # where we're determining key graph relationships
    pivotFrag = pixFrags[i,:,:,:]
    
    for j in range(0,numFragments):
        # don't want to compare to self!
        
        bigBreak = False
        
        if j != i:
            # let's begin in the middle of the two images and
            # try tentative offsets/matching data subsequences from there.
            queryFrag = pixFrags[j,:,:,:]
            
            # for this preliminary trial, we'll pivot on the outermost
            # rows and columns of pivotFrag's data, probing for matching
            # subsequences in queryFrag (where we'll have to look through
            # a non-trivial range of columns and rows (due to overlap/offset))
            
            # declaring bounds for subsequence search
            maxMatchWidth = colInterval//2
            maxMatchHeight = rowInterval//2
            
            maxWidthPush = maxMatchWidth//2
            maxHeightPush = maxMatchHeight//2
            
            # let's start comparing the right side of pivotFrag
            # with the left side of queryFrag.
            midRowPivot = fragsHeight//2
            midColPivot = fragsWidth//2
            
            # looking for small comparison subsets at first (then building out to confirm)
            midPivotMat = (pivotFrag[0,midColPivot-3:midColPivot+3,:]
                           .astype('int32'))
            bigPivotMat = (pivotFrag[0,
                                     midColPivot-maxWidthPush:midColPivot+maxWidthPush,:]).astype('int32')
            
            tempQueryYIdx = fragsHeight-1-tunnelDepth
            
            # regarding limits of the search: we may have to 'tunnel' as much as
            # 3*overlapPix pixels + 2*offsetMax into the neighbor to find a match due to how
            # these overlaps work near the original image boundary.
            
            # NOTE: the situation is different if we only have 2 row partitions!!!
            #       need to push y all the way to 4*overlapPix!
            yOverlapFactor = 3
            if rowPartitions == 2:
                yOverlapFactor = 4
            
            # Note the reversal of roles for x/y offsets here compared to left/right relationships.
            while tempQueryYIdx > fragsHeight-1- (yOverlapFactor*overlapPix + 2 + 2*offsetMax):
                
                tempXOffset = -3*offsetMax
                while tempXOffset < 3*offsetMax + 2:
                    
                    qCS = midColPivot - 3 + tempXOffset
                    qCE = midColPivot + 3 + tempXOffset

                    # what we're creating here is an initial comparison
                    # sliding window of 7 pixels total (3 up, 3 down).
                    # if these are "close enough" to each other, we'll push
                    # the comparison out to try and verify a match.
                    
                    # let's try and anticipate some possible corruption/bias in our
                    # comparison here.
                    tempQueryMat = queryFrag[tempQueryYIdx,qCS:qCE,:].astype('int32')
                    tempDiffMat = abs(tempQueryMat-midPivotMat)
                    
                    # being in uint8, with 3 channels, let's look for similarity
                    # (7*1*3 = 21 comparison pixel window)
                    if sum(sum(tempDiffMat)) < 10*tempDiffMat.size:
                        # in the case of a close preliminary match, we'll expand the
                        # compare to a much bigger window.
                        qCS = midColPivot - maxWidthPush + tempXOffset
                        qCE = midColPivot + maxWidthPush + tempXOffset
                        
                        bigQueryMat = queryFrag[tempQueryYIdx,qCS:qCE,:].astype('int32')    
                        tempDiffMat = abs(bigQueryMat-bigPivotMat)
                        
                        if sum(sum(tempDiffMat)) <= 5*tempDiffMat.size:
                            # in the case of a 'big match' we'll tunnel even further
                            # and see if the match continues.
                            
                            for yTunnel in range(1,tunnelDepth):
                                bigQueryMat = queryFrag[tempQueryYIdx+yTunnel,qCS:qCE,:].astype('int32')
                                bigPivotMatTemp = (pivotFrag[yTunnel,
                                                             midColPivot-maxWidthPush:midColPivot+maxWidthPush,
                                                             :]).astype('int32')
                                tempDiffMat = abs(bigQueryMat-bigPivotMatTemp)
                                
                                if sum(sum(tempDiffMat)) > 2*tempDiffMat.size:
                                    break
                                elif yTunnel == tunnelDepth-1 :
                                    # remember that we're in the left-pivot-to-right-query
                                    # comparison scheme right here.
                                    fragmentGraph[i,2,0] = j
                                    fragmentGraph[i,2,1] = tempXOffset
                                    fragmentGraph[i,2,2] = tempQueryYIdx-fragsHeight
                                    bigBreak = True;
                    
                    # if we've found a match, break from the grind in Y and go
                    # to next pivot fragment (this'll take a couple break statements)
                    if(bigBreak):
                        break
                    else:
                        tempXOffset += 1
                
                # if we've found a match, break from the grind in X tunneling
                if(bigBreak):
                    break
                else:
                    tempQueryYIdx -= 1
            
            # here's the limit of the i/j frag pair testing (if i != j)
        
        # now, IMPORTANT: if we've found a match and reached here, we
        # DON'T want to break but rather CONTINUE to the next pivot fragment.
        if(bigBreak):
            continue

Wall time: 5.75 s


In [25]:
# comparing vs. true graph (with/without offsets)
#print(fragmentGraph)
#print(fragmentGraph[:,0:4,0])
#print(trueGraph)
#print(fragmentGraph[:,0:4,0]-trueGraph)

In [26]:
# Let's now build an ordering for the photos based on everything being done
# correctly (for now).
reducedGraph = fragmentGraph[:,0:4,0]

# finding the top-left element
tlIndex = 0
for i in range(0,numFragments):
    if reducedGraph[i,0] == -1 and reducedGraph[i,2] == -1:
        tlIndex = i
        break
        
# building left to right, row by row
restoredIndices = -1*np.ones(numFragments,dtype='int')
restoredIndices[0] = tlIndex

for i in range(1,numFragments):
    if(i % colPartitions == 0):
        # new row: grabbing index from data above
        restoredIndices[i] = reducedGraph[restoredIndices[i-colPartitions],3]
    else:
        restoredIndices[i] = reducedGraph[restoredIndices[i-1],1]

In [27]:
# Reconstructing image. NOTE: offsets and overlap slightly cheated for now
# for printing purposes.
pixReconstructed = np.zeros((rowInterval*rowPartitions,colInterval*colPartitions,3),dtype = 'uint8')

for i in range(0,numFragments):
    blockRowInd = i // colPartitions
    blockColInd = i % colPartitions
    
    startRow = rowInterval*blockRowInd
    endRow = startRow + rowInterval
    
    startCol = colInterval*blockColInd
    endCol = startCol + colInterval
    
    fragRS = overlapPix
    fragRE = rowInterval + overlapPix

    fragCS = overlapPix
    fragCE = colInterval + overlapPix

    pixReconstructed[startRow:endRow,startCol:endCol,:] = pixFrags[restoredIndices[i],fragRS:fragRE,fragCS:fragCE,:]
    
# row partitions
for i in range(1,rowPartitions):
    rowBegin = i*rowInterval-4
    rowEnd   = i*rowInterval+5
    pixReconstructed[rowBegin:rowEnd,:,:] = 255
        
# column partitions
for j in range(1,colPartitions):
    colBegin = j*colInterval-4
    colEnd   = j*colInterval+5
    pixReconstructed[:,colBegin:colEnd,:] = 255

In [28]:
imReconstructed = Image.fromarray(pixReconstructed)
#imReconstructed